Vamos a armar la red completa con:
    
    * Genes y sus interacciones de StringDB (score > 0.8)
    * Variantes y su mapeo en genes
    * Fenotipos y sus asociaciones con variantes
    * Categorías generales de fenotipos

In [86]:
%run imports.py
%run leer_red.py

assoc = leer_red_df()
print(f"{len(assoc):,} asociaciones")

assoc.sample(10)

/home/juan/miniconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


112,626 asociaciones


,gen_mapeado_elegido,alelo_riesgo,fenotipo,categoria_fenotipo
60359,NaN,rs61416196-A,Blood protein levels,Other measurement
92759,CEP350,rs7540856-C,Parental longevity (mother's age at death),Other measurement
1537,SORCS3,rs1484246-?,General cognitive ability,Biological process
56063,NaN,rs72716801-G,Alcohol use disorder,Other measurement
82320,NaN,rs79757778-A,Lean body mass,Other measurement
34812,LINC01185,rs702873-G,Psoriasis,Immune system disorder
72963,NaN,rs36082259-T,Estimated glomerular filtration rate in non-diabetics,Other measurement
36566,HNF1A,rs2259816-?,C-reactive protein levels,Inflammatory measurement
4988,NaN,rs62422687-A,Intelligence (MTAG),Biological process
81816,GRM8,rs73228917-T,Itch intensity from mosquito bite adjusted by bite size,Other measurement


In [23]:
# Escribo un archivo de fenotipos para elegir a mano
fp = "results/lista_de_fenotipos.csv"

assoc.fenotipo.value_counts().to_csv(fp, index=True, header=False)
print(fp)
!ls -lh $fp

results/lista_de_fenotipos.csv
-rw-r--r-- 1 juan juan 152K Jul 16 16:50 results/lista_de_fenotipos.csv


In [2]:
prot_links = pd.read_table("results/prot_links.stringdb.tsv.gz")
print(f"{len(prot_links):,} links entre proteínas")

prot_links.sample(5)

383,408 links entre proteínas


,protein_1,protein_2
92011,PLA2G7,APOB
72570,COPS4,CUL3
64373,MAPK3,MPRIP
178445,PPP2R5E,PSMA3
261393,TPM2,MYLK


In [3]:
# * Genes y sus interacciones de StringDB (score > 0.8)
enlaces_gen_gen = zip(prot_links.protein_1, prot_links.protein_2)

# * Variantes y su mapeo en genes
enlaces_variante_gen = (
    assoc[["gen_mapeado_elegido", "alelo_riesgo"]]   
    .dropna()
    .drop_duplicates()
    .set_index("alelo_riesgo")
    .gen_mapeado_elegido
    .items()
)

# * Fenotipos y sus asociaciones con variantes
enlaces_variante_fenotipo = (
    assoc[["alelo_riesgo", "fenotipo"]]
    .dropna()
    .drop_duplicates()
    .set_index("alelo_riesgo")
    .fenotipo
    .items()
)
    
# * Categorías generales de fenotipos
enlaces_fenotipo_categoria = (
    assoc[["fenotipo", "categoria_fenotipo"]]
    .dropna()
    .drop_duplicates()
    .set_index("fenotipo")
    .categoria_fenotipo
    .items()
)

red = nx.Graph()

red.add_edges_from(enlaces_gen_gen, tipo="PPI")
red.add_edges_from(enlaces_variante_gen, tipo="mapeo")
red.add_edges_from(enlaces_variante_fenotipo, tipo="asociacion")
red.add_edges_from(enlaces_fenotipo_categoria, tipo="ontologia")

print(f"{len(red):,} nodos en la red")

113,010 nodos en la red


In [4]:
genes_observados = set(prot_links.protein_1) | set(prot_links.protein_2) | set(assoc.gen_mapeado_elegido.dropna())
print(f"{len(genes_observados):,} genes observados")

tipos_de_nodo = dict(
    [(item, "gen") for item in genes_observados] +
    [(item, "alelo") for item in assoc.alelo_riesgo] +
    [(item, "fenotipo") for item in assoc.fenotipo] +
    [(item, "categoria_fenotipo") for item in assoc.categoria_fenotipo]
)

nx.set_node_attributes(red, tipos_de_nodo, name="tipo")

15,988 genes observados


In [72]:
fp = "results/red_completa.gml" 
nx.write_gml(red, fp)

print(fp)
!ls -lh $fp

results/red_completa.gml
-rw-r--r-- 1 juan juan 29M Jul  7 19:50 results/red_completa.gml


## Sub-redes más chicas para que el problema sea tratable

La red completa ocupa decenas de gigas en RAM al transformarla en matriz. Vamos a generar unas subredes más tratables.

In [90]:
# Estos fenos nos parecen interesantes pero estaban categorizados de manera
# muy general, bajo "Biological Process" (bolsa de gatos). Los elegimos a mano:
fenos_mentales = [
    "General cognitive ability",
    "Intelligence (MTAG)",
    "Adventurousness",
    "Automobile speeding propensity",
    "Morning vs. evening chronotype",
    "Suicide attempts",
    "Self-reported risk-taking behaviour",
    "Word reading",
    "Extremely high intelligence",
]

es_feno_mental = assoc.fenotipo.isin(fenos_mentales)
assoc.loc[es_feno_mental, "categoria_fenotipo"] = "Neuro/Cogni/Emo"

categorias_interesantes = [
    "Neurological disorder",
    "Cancer",
    "Cardiovascular disease",
    "Immune system disorder",
    "Metabolic disorder",
    "Digestive system disorder",
    "Neuro/Cogni/Emo"
]

es_categoria_interesante = assoc.categoria_fenotipo.isin(categorias_interesantes)
assoc_elegidas = assoc[es_categoria_interesante]
assoc_a_descartar = assoc[~assoc.index.isin(assoc_elegidas.index)]

print("\n" + "--- TODAS ---" + "\n")
print(assoc.categoria_fenotipo.value_counts())
print("\n" + "--- ELEGIDAS ---" + "\n")
print(assoc_elegidas.categoria_fenotipo.value_counts())


--- TODAS ---

Other measurement                   49746
Hematological measurement            8297
Neurological disorder                5653
Biological process                   5634
Cancer                               5070
Body measurement                     4815
Lipid or lipoprotein measurement     4655
Other trait                          4599
Other disease                        3923
Response to drug                     3670
Immune system disorder               2880
Cardiovascular measurement           2799
Cardiovascular disease               2782
Neuro/Cogni/Emo                      2257
Metabolic disorder                   2236
Digestive system disorder            1725
Inflammatory measurement             1499
Liver enzyme measurement              321
Name: categoria_fenotipo, dtype: int64

--- ELEGIDAS ---

Neurological disorder        5653
Cancer                       5070
Immune system disorder       2880
Cardiovascular disease       2782
Neuro/Cogni/Emo              2257


In [91]:
subred = red.copy()

# Primero quitamos los fenotipos y categorías no elegidas.
subred.remove_nodes_from(assoc_a_descartar.fenotipo)

# Luego los alelos que no estén asociados a esos fenotipos.
alelos_a_descartar = assoc.alelo_riesgo[~assoc.alelo_riesgo.isin(assoc_elegidas.alelo_riesgo)]
subred.remove_nodes_from(alelos_a_descartar)

# No descartamos genes porque forman parte de la red de PPI y pueden influir
# indirectamente en el label spreading, aun sin estar conectados directamente
# a un alelo con asociación.

nodos_CG = max(nx.connected_components(subred), key=len)
subred_CG = nx.subgraph(subred, nodos_CG)

print(f"{len(red):,} nodos en la red completa")
print(f"{len(subred_CG):,} nodos en la subred")

fp = "results/subred.gml"
nx.write_gml(subred, fp)
print(fp)
!ls -lh $fp

fp = "results/subred.tsv"
assoc_elegidas.to_csv(fp, sep="\t", index=False)
print(fp)
!ls -lh $fp

113,010 nodos en la red completa
35,112 nodos en la subred
results/subred.gml
-rw-r--r-- 1 juan juan 16M Jul 19 18:49 results/subred.gml
results/subred.tsv
-rw-r--r-- 1 juan juan 1.4M Jul 19 18:49 results/subred.tsv
